## Toggle Flip-Flop

In this example we create a toggle flip-flop from a d-flip-flop and an xor gate. This shows how to construct sequential logic circuits using `Magma`.

In [1]:
import magma as m

import os
os.environ['MANTLE_TARGET'] = 'ice40'
from mantle import DFF

import mantle lattice ice40
import mantle lattice mantle40


In [2]:
def tff(I):
    # instance a dff to hold the state of the toggle flip-flop - this needs to be done first
    ff = DFF()

    # compute the next state using the input I and the old state ff.O
    ff( I ^ ff.O)

    return ff.O

In [3]:
from loam.boards.icestick import IceStick

icestick = IceStick()
icestick.Clock.on()
icestick.J1[0].rename('J1').input().on()  # rename so it is not an array
icestick.J3[0].rename('J3').output().on()

main = icestick.DefineMain()
m.wire( tff(main.J1), main.J3 )
m.EndDefine()

## Compile and Build

In [4]:
m.compile("build/tff", main)

compiling XOr2
compiling main


In [5]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif tff.blif' tff.v
arachne-pnr -q -d 1k -o tff.txt -p tff.pcf tff.blif 
icepack tff.txt tff.bin
# iceprog tff.bin

/Users/hanrahan/git/magmathon/notebooks/build


If you connect a switch to `J1`,
then the flip-flop should toggle when the switch is on.

Here is the verilog.

In [6]:
with open("build/tff.v", "r") as tff_verilog:
    print(tff_verilog.read())

module XOr2 (input [1:0] I, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h6666)) inst0 (.I0(I[0]), .I1(I[1]), .I2(1'b0), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module main (input  J1, output  J3, input  CLKIN);
wire  inst0_Q;
wire  inst1_O;
SB_DFF inst0 (.C(CLKIN), .D(inst1_O), .Q(inst0_Q));
XOr2 inst1 (.I({inst0_Q,J1}), .O(inst1_O));
assign J3 = inst0_Q;
endmodule




Note that this example uses the built-in ICE40 `SB_DFF` and `SB_LUT4` primitives.